In [1]:
import pandas as pd
import numpy as np
import re
import datetime


In [2]:
dataset_name = "raw_cdr_data.csv"

raw_cdr_data = pd.read_csv(dataset_name, header=None, low_memory=False)


In [5]:
#====================Functions for Preprocessing the data==============#

# Function to separate the date and time from column 9
def datetime_divider(data):
    # data type of data is list
    # ["20190620032717.906"]
    # return type will be list of list with date, time [["20190620","032717"]]
    
    for index in range(len(data)):
        #find the string which starts with only digit
        if re.match('\d',str(data[index])):
            regex = re.compile("\d{1,8}") 
            a = regex.findall(str(data[index]))
            data[index] = [a[0],a[1]] #date and time
            
        else:
            data[index] = [np.nan, np.nan]
    
    return data
            

# Function to convert the date to desired format
def date_modifier(data):
    # data type of data is list
    # 20190620 should be converted to 2019-06-20
    for index in range(len(data)):
        if re.match('\d',str(data[index])):
            year = data[index][:4]
            month = data[index][4:6]
            day = data[index][6:]
            
            data[index] = "-".join([year,month,day])
        
        else:
            data[index] = np.nan
    
    return data

# Function to convert the time to desired timme format
def time_modifier(data):
    # data type of data is list
    # 032717 should be converted into 03:27:17 AM
    # COnvert 24 hr format to 12 hr format
    for index in range(len(data)):
        if re.match('\d',str(data[index])):
            hours = int(data[index][:2])
            minutes = data[index][2:4]
            seconds = data[index][4:]
            
            if hours >= 12:
                if hours == 12:
                    hr = str(hours)
                else:
                    hr = str(hours-12)
                    
                meridiem = "PM"
            
            else:
                if hours == 0:
                    hr = str(12)
                else:
                    hr = str(hours)
                meridiem = "AM"
            
            data[index] = ":".join([hr,minutes,seconds])+ " " + meridiem
            
        else:
            data[index] = np.nan
            
    return data


# Function to replace the terminologies
def replace_simple_with_standard_terminology(dataframe):
    dataframe[5] = dataframe[5].replace('Originating','Outgoing')
    dataframe[5] = dataframe[5].replace('Terminating','Incoming')
    
    dataframe[267] = dataframe[267].replace('Success','Voice Portal')
    
    dataframe[312] = dataframe[312].replace('Shared Call Appearance','Secondary Device')
    
    return dataframe



def remove_unwanted_data(datacolumn):
    for index in range(len(datacolumn)):
        if (datacolumn[index]== 'Secondary Device' or datacolumn[index]=='Primary Device'):
            continue
        else:
            datacolumn[index] = np.nan
            
    return datacolumn


# This function sets all the services in one column 147 for 147,312,267 columns
def combine_all_services(datacolumn147, datacolumn312, datacolumn267):
    for index in range(len(datacolumn147)):
        if datacolumn147[index] is np.nan:
            if (datacolumn312[index] is not np.nan) and (datacolumn267[index] is not np.nan):
                datacolumn147[index] = str(datacolumn312[index]) +',' + str(datacolumn267[index])
            elif (datacolumn312[index] is not np.nan):
                datacolumn147[index] = datacolumn312[index]
            else:
                datacolumn147[index] = datacolumn267[index]
               
        else:
            continue
        
    return datacolumn147


# Convert the datetime into desired format
# "20190620032717.906"  to "2019-06-20 03:27:17"
def call_time_fetcher(data):
    
    for index in range(len(data)):
        data[index] = str(data[index])
        if (data[index]!='nan'):
            year = data[index][:4]
            month = data[index][4:6]
            day = data[index][6:8]
            hours = data[index][8:10]
            minutes = data[index][10:12]
            seconds =str(round(float(data[index][12:])))
            
            if( int(seconds) >= 60):
                minutes = int(minutes) + 1 
                seconds = int(seconds) - 60
                
            if( int(minutes) >= 60):
                hours = int(hours) + 1
                minutes = int(minutes) - 60
                
            data[index] = f"{year}-{month}-{day} {hours}:{minutes}:{seconds}"
        
        else:
            data[index] = np.nan
    
    return data


# ['3:27:17 AM', '1:28:19 PM', 'nan', '7:23:52 PM']
# ['3:00 - 3:59', '13:00 - 13:59', nan, '19:00 - 19:59']
def hourly_range(data):
    
    for index in range(len(data)):
        data[index] = str(data[index])
        
        if(data[index] != 'nan'):
            if(re.search("PM",data[index])):
                time_data = re.findall('\d+',data[index])
                if time_data[0] !="12":
                    time_data = int(time_data[0])+12
                else:
                    time_data = time_data[0]
            
            else:
                time_data = re.findall('\d+', data[index])
                if time_data[0]=='12':
                    time_data = f"0{int(time_data[0]) -12}"
                else:
                    time_data = time_data[0]
            
            
            data[index] = f"{time_data}:00 - {time_data}:59"
  
        else:
            data[index] = np.nan
    
    return data
    

# ['2019-06-20', '2019-06-21', 'nan', '2019-06-25']
# ['Thursday', 'Friday', nan, 'Tuesday']
def weekly_range(data):
    
    for index in range(len(data)):
        data[index] = str(data[index])
        
        if data[index] != 'nan':
            year, month, day = [int(x) for x in data[index].split("-")] #list unpacking
            result = datetime.date(year,month,day)
            data[index] = result.strftime("%A") #full weekday name
        
        else:
            data[index] = np.nan
    
    return data


#===========================END=======================================#


In [6]:
raw_cdr_data["starttime"] = pd.to_datetime(call_time_fetcher(raw_cdr_data[9].tolist()))

In [11]:
(raw_cdr_data["starttime"] [3]-raw_cdr_data["starttime"] [1])

Timedelta('0 days 02:00:17')